# Notebook 1: The Building Blocks - Environment & PyTorch Tensors

In this notebook, we'll set up a clean, modern Python environment and master the fundamental data object in all of deep learning: the tensor. Understanding tensors is crucial—they are the foundation upon which all neural network operations are built.


## Platform Disclaimer

**Important:** This tutorial is designed for macOS, Linux, and Windows. The tooling (uv) is optimized for macOS and Linux. For GPU acceleration, PyTorch supports:
- **CUDA** (NVIDIA GPUs on Linux/Windows)
- **MPS** (Apple Silicon GPUs on macOS)
- **CPU** (fallback for all platforms)

The code will automatically detect and use the best available device.


## Environment Setup

We'll use `uv` as a modern, extremely fast replacement for pip and venv. It's designed to be a drop-in replacement that's orders of magnitude faster.

Here are the three steps for setup:

1. **Create a virtual environment:** A self-contained sandbox for our project's libraries.
2. **Activate the environment:** "Step inside" the sandbox.
3. **Install packages:** Use `uv pip install` to add our tools.

Run these commands in your terminal:

```bash
# Create a virtual environment
uv venv

# Activate the environment (macOS/Linux)
source .venv/bin/activate

# Install required packages
uv pip install torch numpy jupyter
```


## Introduction to Tensors

What is a tensor? In the simplest terms: **It's a multi-dimensional array, like a list of lists of lists. Think of it as a super-powered NumPy array that can run on GPUs.**

Tensors are the core data structure in PyTorch. We can create them in several ways:
- From Python lists
- From NumPy arrays
- Using built-in functions like `torch.rand()`, `torch.ones()`, `torch.zeros()`
- With specific values using `torch.tensor()`


In [ ]:
import torch
import numpy as np

# Create tensor from a Python list
tensor_from_list = torch.tensor([1, 2, 3, 4, 5])
print("From list:", tensor_from_list)

# Create tensor from a NumPy array
numpy_array = np.array([1.0, 2.0, 3.0])
tensor_from_numpy = torch.from_numpy(numpy_array)
print("From NumPy:", tensor_from_numpy)

# Create random tensor
random_tensor = torch.rand(3, 4)
print("Random tensor:\n", random_tensor)

# Create tensor of ones
ones_tensor = torch.ones(2, 3)
print("Ones tensor:\n", ones_tensor)

# Create tensor of zeros
zeros_tensor = torch.zeros(2, 3)
print("Zeros tensor:\n", zeros_tensor)


## Tensor Attributes

Every tensor has three critical attributes that you'll constantly check:

1. **`.shape`**: The dimensions of the tensor. This is the most critical concept for debugging models—mismatched shapes are the #1 cause of errors.
2. **`.dtype`**: The data type (e.g., `float32`, `int64`, `long`). Determines how much memory is used and what operations are allowed.
3. **`.device`**: Where the tensor lives (e.g., `'cpu'`, `'cuda'`, or `'mps'`). This determines whether computations run on CPU or GPU.


In [ ]:
# Create a tensor and inspect its attributes
example_tensor = torch.rand(3, 4, 5)

print("Tensor shape:", example_tensor.shape)
print("Tensor dtype:", example_tensor.dtype)
print("Tensor device:", example_tensor.device)
print("\nFull tensor:\n", example_tensor)


## The Art of Shaping Data

Tensor operations are the core skill for a code-first deep learning practitioner. Being able to manipulate and reshape tensors is essential for preparing data for your models.

Standard indexing and slicing works just like NumPy—if you're familiar with NumPy, you already know how to do this in PyTorch.


In [ ]:
# Create a 4x4 tensor
matrix = torch.arange(16).reshape(4, 4)
print("Original 4x4 tensor:\n", matrix)

# Slice the first row
first_row = matrix[0, :]
print("\nFirst row:", first_row)

# Slice the first column
first_column = matrix[:, 0]
print("First column:", first_column)

# Slice the last column
last_column = matrix[:, -1]
print("Last column:", last_column)


## Reshaping with view()

The `.view()` method is incredibly powerful—it reshapes a tensor without copying the data. It's a "free" operation that just creates a new view of the same underlying data.

The `-1` trick is particularly useful: it tells PyTorch to automatically infer that dimension based on the total number of elements. For example, if you have a tensor with 12 elements and call `.view(3, -1)`, PyTorch will automatically make it `(3, 4)`.


In [ ]:
# Create a tensor with initial shape
original = torch.arange(12)
print("Original shape:", original.shape)
print("Original tensor:", original)

# Reshape using view with -1 trick
reshaped = original.view(2, -1)
print("\nReshaped to (2, -1):\n", reshaped)
print("New shape:", reshaped.shape)


## Adding and Removing Dimensions

`unsqueeze()` and `squeeze()` are essential for managing tensor dimensions. The most common practical use case: **You will use `unsqueeze(0)` constantly to add a 'batch' dimension to a single data point before feeding it to a model.**

- `unsqueeze(dim)`: Adds a dimension of size 1 at the specified position
- `squeeze(dim)`: Removes a dimension of size 1 at the specified position (or all size-1 dimensions if no dim specified)


In [ ]:
# Create a tensor
tensor = torch.tensor([1, 2, 3, 4, 5])
print("Original shape:", tensor.shape)
print("Original tensor:", tensor)

# Add a dimension at position 0 (adds batch dimension)
with_batch = tensor.unsqueeze(0)
print("\nAfter unsqueeze(0):")
print("Shape:", with_batch.shape)
print("Tensor:\n", with_batch)

# Remove the dimension we just added
back_to_original = with_batch.squeeze(0)
print("\nAfter squeeze(0):")
print("Shape:", back_to_original.shape)
print("Tensor:", back_to_original)


## CPU vs. GPU Acceleration

Large computations are much faster on a GPU. PyTorch supports multiple GPU backends:
- **CUDA**: For NVIDIA GPUs (Linux/Windows)
- **MPS**: For Apple Silicon GPUs (macOS)
- **CPU**: Fallback for all platforms

In the next cell, we'll:
1. Detect the best available device (CUDA → MPS → CPU)
2. Create two large tensors
3. Time a matrix multiplication on the CPU
4. Move them to the GPU device using `.to(device)` (if available)
5. Time the same operation on GPU

You should see a significant speedup on GPU for large operations!


In [ ]:
import time

# Detect the best available device: CUDA → MPS → CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available! Using Apple Silicon GPU.")
else:
    device = torch.device("cpu")
    print("No GPU available. Using CPU.")

size = 2000
a_cpu = torch.rand(size, size)
b_cpu = torch.rand(size, size)

# Time CPU computation
start_time = time.time()
result_cpu = torch.matmul(a_cpu, b_cpu)
cpu_time = time.time() - start_time
print(f"\nCPU time: {cpu_time:.4f} seconds")

# Time GPU computation if available
if device.type != "cpu":
    a_gpu = a_cpu.to(device)
    b_gpu = b_cpu.to(device)
    
    # Warm up (first operation can be slower)
    _ = torch.matmul(a_gpu, b_gpu)
    
    start_time = time.time()
    result_gpu = torch.matmul(a_gpu, b_gpu)
    gpu_time = time.time() - start_time
    print(f"{device.type.upper()} time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x faster")
else:
    print("No GPU available - skipping GPU comparison")
